In [10]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
#
#Carregar o csv e converter para parquet
meuSchema = 'timestamp DATE, user_id STRING, latitude FLOAT, longitude FLOAT'
#df = spark.read.csv("file:///Users/alexa/Desktop/spark/location_record.csv",header='True', schema=meuSchema)
df = spark.read.csv("data/location_record.csv",header='True', schema=meuSchema)
df.write.mode('overwrite').parquet("csv_to_parquet")
df = spark.read.parquet("csv_to_parquet")
df.show(5)
df.count()

+----------+--------------------+---------+-----------+
| timestamp|             user_id| latitude|  longitude|
+----------+--------------------+---------+-----------+
|2017-02-23|ceb41a9b-a2ae-4fa...|  33.5851| -112.38525|
|2017-07-07|368d2b49-51e3-45e...| 41.89339|  -88.03215|
|2017-11-22|d9ca58e8-a5f2-4e5...|  33.6151|-112.375244|
|2017-11-17|c6b3afe0-2b6c-4bb...| 32.81424| -117.39917|
|2017-03-22|3248fb1c-4218-433...|29.451138|   -98.7646|
+----------+--------------------+---------+-----------+
only showing top 5 rows



1460950

In [2]:
#Quantidade de registros de localização por user_id
#
#df_agregado = df.groupBy('user_id').agg({'timestamp': 'count'}).show()
df_agregado = df.groupBy('user_id').count().show()

+--------------------+-----+
|             user_id|count|
+--------------------+-----+
|6bee6eeb-3de9-428...|18058|
|2930195c-571f-423...|11720|
|ceb41a9b-a2ae-4fa...|17449|
|956f008e-afc4-4c2...|14827|
|ca2004f5-4cb1-47c...|17453|
|9766d90d-02a9-437...|13488|
|ea01cebf-6020-4f8...|10746|
|f38ac15b-5b53-40b...|19670|
|20d0e40a-fac7-40d...|11064|
|38d39b3d-333a-4cf...|10461|
|bf6239ad-5d97-45e...|17537|
|29285d0b-d2ac-45e...|17643|
|ee31a564-abce-4b9...|11297|
|f6b1adf3-7fe1-4ee...|12719|
|7e8bf208-d924-4a1...|11811|
|887d38e3-1425-4c8...|11783|
|7196e0b5-74e4-49d...|11306|
|02fa7c31-61f3-440...|15518|
|9377f083-89e7-421...|11799|
|f4ad21f9-1aea-41c...|18564|
+--------------------+-----+
only showing top 20 rows



In [3]:
#Quantidade de registros do user_id por data
#
df.groupBy('user_id','timestamp').agg({'timestamp': 'count'}).show()

+--------------------+----------+----------------+
|             user_id| timestamp|count(timestamp)|
+--------------------+----------+----------------+
|5e96b5ba-57f6-4c9...|2017-05-22|              35|
|2f6c0454-0419-40b...|2017-11-16|              41|
|0c6c3ab9-c06c-41e...|2017-07-18|              43|
|6fca3db0-8e41-429...|2017-02-14|              50|
|debc30cc-be75-4db...|2017-01-08|              42|
|e4abaf63-5618-489...|2017-01-22|              41|
|f4ad21f9-1aea-41c...|2017-10-12|              64|
|7a9784be-2b75-42f...|2017-08-10|              41|
|0c6c3ab9-c06c-41e...|2017-06-26|              45|
|04d50310-688f-4b0...|2017-05-29|              39|
|b98df091-0375-495...|2017-11-06|              43|
|ca2004f5-4cb1-47c...|2017-03-18|              58|
|91404c28-4877-4e4...|2017-08-12|              53|
|23db51bb-2fa1-4e5...|2017-04-02|              48|
|8d8d47d3-d3ea-4f6...|2017-12-09|              24|
|c8270c5e-8b30-4a0...|2017-09-29|              33|
|f4ad21f9-1aea-41c...|2017-05-1

In [7]:
#Filtrar dados de apenas 1 user_id
#
#df2 = df[df['user_id'] == 'ca2004f5-4cb1-47c6-b1cf-43381cd782db']
df2 = df[df['user_id'] == 'ceb41a9b-a2ae-4fa2-9476-f308c775e8f6']
#df2 = df[df['user_id'] == '368d2b49-51e3-45e2-ba4f-13285435b3f8']
#
#Modificar o dataframe para conter apenas os dados de localização
#
df2 = df2[['latitude','longitude']]
df2.printSchema()
df2.show(5)
df2.count()

root
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)

+--------+-----------+
|latitude|  longitude|
+--------+-----------+
| 33.5851| -112.38525|
| 33.6151|-112.375244|
| 33.5751|-112.375244|
| 33.6051| -112.40524|
| 33.6151| -112.35525|
+--------+-----------+
only showing top 5 rows



17449

In [8]:
#import numpy as np
#import pandas as pd
# ativa transfencia de dados colunados via Arrow
#spark.conf.set("spark.sql.execution.arrow.enabled", "true")
dfpandas = df2.select("*").toPandas()

In [9]:
import gmaps
#import gmaps.datasets
gmaps.configure(api_key="AIza...")
#mapa = gmaps.figure(map_type='SATELLITE')
figure_layout = { 'width': '100%', 'height': '500px', 'border': '1px solid black', 'padding': '1px'} 
mapa = gmaps.figure(map_type='HYBRID',layout=figure_layout)  
mapa.add_layer(gmaps.heatmap_layer(dfpandas))
#mapa.add_Layer(gmaps.directiopns.Directions(dfpandas, SF, mode='car'))
mapa

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='1px', width='100%'))